In [1]:
import os
import numpy as np
import pandas as pd

# Get Relevant information from PLIC file

In [2]:
def GetClassData(df_Complete, Class_ID, Term, Nbr):
    df = df_Complete[df_Complete['Class_ID'] == Class_ID]
    df['Academic Term Sdescr'] = Term
    df['Catalog Nbr'] = Nbr
    df.loc[df['Survey_x'] == 'F', 'PreScores'] = np.nan
    df.loc[df['Survey_y'] == 'F', 'PostScores'] = np.nan
    df = df[~(pd.isnull(df['PreScores'])) | ~(pd.isnull(df['PostScores']))].reset_index(drop = True)
    df['Q5a_x'] = df['Q5a_x'].astype(str).str.split('@').str.get(0).str.lower()
    df['Q5a_y'] = df['Q5a_y'].astype(str).str.split('@').str.get(0).str.lower()
    return df[['Q5a_x', 'Q5a_y', 'PreScores', 'PostScores', 'Academic Term Sdescr', 'Catalog Nbr']]

In [3]:
PLIC_Complete = pd.read_csv('C:/Users/Cole/Documents/PLIC_DATA/Collective_Surveys/Complete/Complete_Concat.csv')

Fall2017_1112 = GetClassData(PLIC_Complete, 'R_2xOT2Y1NtNiseCk', '2017FA', '1112')
Fall2017_2213 = GetClassData(PLIC_Complete, 'R_zfk080BHz6RWixb', '2017FA', '2213')
Fall2017_1116 = GetClassData(PLIC_Complete, 'R_1Oko8BpPfb9rt0G', '2017FA', '1116')
Fall2017_2217 = GetClassData(PLIC_Complete, 'R_12QFe4VQPh6oNW1', '2017FA', '2217')

Spring2018_1112 = GetClassData(PLIC_Complete, 'R_1LHvn3R5Afj8eUc', '2018SP', '1112')
Spring2018_1116 = GetClassData(PLIC_Complete, 'R_2R8MnTyv2jFgPzA', '2018SP', '1116')

Fall2018_1112 = GetClassData(PLIC_Complete, 'R_3ijRcPfXo8MUfFj', '2018FA', '1112')
Fall2018_1116 = GetClassData(PLIC_Complete, 'R_1IB300CxBKh0Tw7', '2018FA', '1116')

Spring2019_1112 = GetClassData(PLIC_Complete, 'R_RKRNIWFu1gZuSPf', '2019SP', '1112')

df = pd.concat([Fall2017_1112, Fall2017_2213, Fall2017_1116, Fall2017_2217, Spring2018_1112, Spring2018_1116, Fall2018_1112,
                Fall2018_1116, Spring2019_1112]).reset_index(drop = True)

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (17,33,34,39,40,65,66,67,68,97,98,101,125,126,135,136,141,142,167,168,171,172,203,204,205,217,218,239,240,241,249,269,270,305,306,311,312,313,323,349,350,351,353,355,357,361,365,385,386,405,406,413,414,419,537,545,547,549,551) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

# Merge with Registrar Data

In [4]:
df_Registrar = pd.read_excel('Registrar_Data.xls').drop(columns = ['ACT | Combined English/Writing', 'ACT | English', 
                                                                   'ACT | Reading', 'ACT | Writing', 
                                                                   'ACT | Writing Subject Score 9/15', 'SAT I | Critical Reading', 
                                                                   'SAT I | Read/Writing Sect Score', 'SAT I | Writing Score'])
df_Registrar[['Catalog Nbr', 'Employee Id']] = df_Registrar[['Catalog Nbr', 'Employee Id']].astype(str)
df_Registrar['Academic_Term_Number'] = df_Registrar['Academic Term Sdescr'].map({'2017FA':1, '2018SP':2, '2018FA':3, '2019SP':4})
df_Registrar = df_Registrar.sort_values(by = 'Academic_Term_Number').drop_duplicates(subset = ['Employee Id', 
                                                                                               'Catalog Nbr'], keep = 'last')

df1 = pd.merge(df, df_Registrar, left_on = ['Q5a_x', 'Academic Term Sdescr', 'Catalog Nbr'],
               right_on = ['Netid', 'Academic Term Sdescr', 'Catalog Nbr'], how = 'inner')
df2 = pd.merge(df, df_Registrar, left_on = ['Q5a_x', 'Academic Term Sdescr', 'Catalog Nbr'],
               right_on = ['Employee Id', 'Academic Term Sdescr', 'Catalog Nbr'], how = 'inner')
df3 = df.merge(df_Registrar, left_on = ['Q5a_y', 'Academic Term Sdescr', 'Catalog Nbr'],
               right_on = ['Netid', 'Academic Term Sdescr', 'Catalog Nbr'], how = 'inner')
df4 = df.merge(df_Registrar, left_on = ['Q5a_y', 'Academic Term Sdescr', 'Catalog Nbr'],
               right_on = ['Employee Id', 'Academic Term Sdescr', 'Catalog Nbr'], how = 'inner')

df_merged = pd.concat([df1, df2, df3, df4]).drop_duplicates(subset = ['Employee Id', 'Academic Term Sdescr'],
                                                            keep = 'last').reset_index(drop = True)

index_vals = [tuple(v) for v in df_merged[['Employee Id', 'Academic Term Sdescr']].values]
filtered = df_Registrar.loc[(df_Registrar['Academic Term Sdescr'] + 
                             df_Registrar['Catalog Nbr']).isin(df_merged['Academic Term Sdescr'] + 
                                                               df_merged['Catalog Nbr'])].set_index(['Employee Id',
                                                                                                     'Academic Term Sdescr'])
filtered_out = filtered.loc[~filtered.index.isin(index_vals)].reset_index()

df_out = pd.concat([df_merged, filtered_out]).drop(columns = ['Q5a_x', 'Q5a_y']).reset_index(drop = True)
df_out = df_out.loc[:, list(df_Registrar.columns) + ['PreScores', 'PostScores']]
df_out['Assessment'] = 'PLIC'
df_out.to_csv('PLICpRegistrar.csv', index = False)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

